## Python для сбора данных

*Алла Тамбовцева, НИУ ВШЭ*

### Задание 1

Используя код с лекции, напишите функцию, которая принимает на вход ссылку на страницу новости на сайте [nplus1.ru](https://nplus1.ru/) и возвращает список из следующих элементов:

* название новости;
* краткое описание новости;
* имя автора новости;
* рубрики, к которым относится новость;
* сложность текста новости;
* текст новости.

Используя код с лекции, получаем:

In [2]:
import requests
from bs4 import BeautifulSoup

def get_info(url0):
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text, 'lxml')
    author = soup0.find_all('meta', 
                            {'name' : 'author'})[0]['content']
    title = soup0.find_all('meta', 
                           {'property' : 'og:title'})[0]['content']
    description = soup0.find_all('meta', 
                             {'property' : 'og:description'})[0]['content']
    date = soup0.find_all('meta', 
                      {'itemprop' : 'datePublished'})[0]['content']
    difficult = soup0.find_all('span', 
                           {'class' : 'difficult-value'})[0].text
    raw_rubrics = soup0.find_all('p', 
                             {'class' : 'table'})[0].find_all('a')
    rubrics = []
    for r in raw_rubrics:
        rubrics.append(r.text)
        
    pars = []
    for p in soup0.find_all('p', {'class' : None}):
        pars.append(p.text)
    text = " ".join(pars)
    
    text_final = text.split("Нашли опечатку?")[0].replace('\xa0', 
                                                      ' ').replace('\n', ' ')
    return [author, title, description, date, difficult, 
           rubrics, text_final] 

### Задание 2

Создайте список ссылок на новости с главной страницы сайта [nplus1.ru](https://nplus1.ru/), примените написанную в задании 1 функцию к каждой ссылке в списке и сформируйте новый список списков с данными по новостям. 

Список ссылок тоже был создан на лекции:

In [4]:
url = "https://nplus1.ru/"
page = requests.get(url)
soup = BeautifulSoup(page.text)

urls = []
for link in soup.find_all('a'):
    if '/news' in link.get('href'):
        urls.append(link.get('href'))
        
full_urls = ['https://nplus1.ru' + u for u in urls]

Но давайте не будем спешить: импортируем функцию `sleep` для задержки, чтобы на каждой итерации цикла, прежде чем перейти к следующей новости, Python ждал несколько секунд. Во-первых, это нужно, чтобы сайт «не понял», чтобы мы его грабим, да еще автоматически. Во-вторых, с небольшой задержкой всегда есть гарантия, что страница прогрузится (сейчас это пока не очень важно, но особенно актуально будет, когда будем обсуждать встраивание в браузер с Selenium). Приступим.

In [5]:
from time import sleep

data = []
for u in full_urls:
    res = get_info(u)
    data.append(res)
    print(u + "...done")
    sleep(0.5)  # задержка 0.5 секунд

https://nplus1.ru/news/2020/04/27/venus-atmosphere-tidal-waves...done
https://nplus1.ru/news/2020/04/27/zumwalt...done
https://nplus1.ru/news/2020/04/27/starship-pressure-passed...done
https://nplus1.ru/news/2020/04/27/e-fan-x...done
https://nplus1.ru/news/2020/04/25/hyperphagia-neurons...done
https://nplus1.ru/news/2020/04/25/dna-pocket...done
https://nplus1.ru/news/2020/04/25/moon-landing-spray...done
https://nplus1.ru/news/2020/04/25/thermogalvanic-hydrogel-cooling...done
https://nplus1.ru/news/2020/04/24/30-years-hubble...done
https://nplus1.ru/news/2020/04/24/feline-grimace-scale...done
https://nplus1.ru/news/2020/04/24/supermassive-black-hole-escape-star...done
https://nplus1.ru/news/2020/04/24/adobe...done
https://nplus1.ru/news/2020/04/24/broken-hearts...done
https://nplus1.ru/news/2020/04/24/00s-taxi...done
https://nplus1.ru/news/2020/04/24/cold-sweet...done
https://nplus1.ru/news/2020/04/24/avian...done
https://nplus1.ru/news/2020/04/24/kentaurs-interstellar...done
https://np

### Задание 3

Преобразуйте полученный в задании 2 список в датафрейм, добавьте названия столбцов, при необходимости измените тип столбцов. Выгрузите полученный датафрейм в файл Excel.

In [6]:
import pandas as pd

df = pd.DataFrame(data)
df.columns = ['author', 'title', 'description', 'date', 'difficult', 
           'rubrics', 'text']

df['difficult'] = df['difficult'].astype(float)
df.to_excel('nplus1.xlsx')